# FalkorDB as Graph Memory

## Prerequisites

### Install Mem0 with Graph Memory support

To use Mem0 with Graph Memory support, install it using pip:

```bash
pip install "mem0ai[graph]"
```

This command installs Mem0 along with the necessary dependencies for graph functionality.

### FalkorDB setup

FalkorDB is a Redis-compatible graph database. You can run it using Docker:

```bash
docker run -p 6379:6379 -it --rm falkordb/falkordb:latest
```

Or install and run it locally following the [FalkorDB installation guide](https://docs.falkordb.com/setup.html).


## Configuration

Do all the imports and configure OpenAI (enter your OpenAI API key):

In [ ]:
from mem0 import Memory
from openai import OpenAI

import os

os.environ["OPENAI_API_KEY"] = ""
openai_client = OpenAI()

Set up configuration to use the embedder model and FalkorDB as a graph store:

In [ ]:
config = {
    "embedder": {
        "provider": "openai",
        "config": {"model": "text-embedding-3-large", "embedding_dims": 3072},
    },
    "graph_store": {
        "provider": "falkordb",
        "config": {
            "host": "localhost",
            "port": 6379,
            "graph_name": "social_network"
        },
    },
}
memory = Memory.from_config(config_dict=config)

## Alternative configuration with authentication

If your FalkorDB instance requires authentication:

In [ ]:
config_with_auth = {
    "embedder": {
        "provider": "openai",
        "config": {"model": "text-embedding-3-large", "embedding_dims": 3072},
    },
    "graph_store": {
        "provider": "falkordb",
        "config": {
            "host": "localhost",
            "port": 6379,
            "username": "default",
            "password": "your_password",
            "graph_name": "social_network"
        },
    },
}
# Uncomment to use authenticated connection
# memory = Memory.from_config(config_dict=config_with_auth)

## Add some memories

Let's create some memories about users and their preferences:

In [ ]:
# Define user data
messages_alice = [
    {
        "role": "user",
        "content": "Hi, I'm Alice, I work at a tech startup in San Francisco. I love playing tennis and hiking."
    },
    {
        "role": "assistant", 
        "content": "Hello Alice! Nice to meet you. I've noted that you work at a tech startup in San Francisco and enjoy tennis and hiking."
    }
]

messages_bob = [
    {
        "role": "user",
        "content": "I'm Bob, Alice's colleague. We both work at the same tech startup. I love playing tennis too, and we sometimes play together."
    }
]

messages_charlie = [
    {
        "role": "user",
        "content": "Hi! I'm Charlie. I'm Alice's friend from her hiking group. We go hiking in the Bay Area trails every weekend."
    }
]

# Add memories
memory.add(messages_alice, user_id="alice")
memory.add(messages_bob, user_id="bob")
memory.add(messages_charlie, user_id="charlie")

## Retrieve memories

Now let's search for memories related to specific queries:

In [ ]:
# Search for tennis-related memories
tennis_memories = memory.search("tennis", user_id="alice")
print("Tennis-related memories for Alice:")
for i, memory_item in enumerate(tennis_memories, 1):
    print(f"{i}. {memory_item['memory']}")

print("\n" + "="*50 + "\n")

# Search for work-related memories
work_memories = memory.search("tech startup work", user_id="bob")
print("Work-related memories for Bob:")
for i, memory_item in enumerate(work_memories, 1):
    print(f"{i}. {memory_item['memory']}")

print("\n" + "="*50 + "\n")

# Search for hiking-related memories
hiking_memories = memory.search("hiking trails", user_id="charlie")
print("Hiking-related memories for Charlie:")
for i, memory_item in enumerate(hiking_memories, 1):
    print(f"{i}. {memory_item['memory']}")

## View the Graph Structure

Let's see what relationships were extracted and stored in the graph:

In [ ]:
# Get all relationships for each user
for user_id in ["alice", "bob", "charlie"]:
    print(f"\nRelationships for {user_id}:")
    all_memories = memory.get_all(user_id=user_id)
    
    if 'graph_entities' in all_memories:
        for entity in all_memories['graph_entities'][:10]:  # Show first 10 relationships
            print(f"  {entity['source']} --[{entity['relationship']}]--> {entity['target']}")
    else:
        print("  No graph entities found")
        
    print(f"  Total memories: {len(all_memories.get('memories', []))}")

## Cross-user Search

The power of graph memory is in discovering connections across users. Let's search for connections:

In [ ]:
# Search for Alice's connections with tennis
print("Alice's tennis connections:")
alice_tennis = memory.search("Alice tennis colleague", user_id="alice")
for memory_item in alice_tennis:
    print(f"- {memory_item['memory']}")

print("\n" + "="*50 + "\n")

# Search for Alice's hiking connections  
print("Alice's hiking connections:")
alice_hiking = memory.search("Alice hiking friend weekend", user_id="alice")
for memory_item in alice_hiking:
    print(f"- {memory_item['memory']}")

print("\n" + "="*50 + "\n")

# Search for shared interests
print("Shared interests and connections:")
shared = memory.search("tech startup colleagues friends activities", user_id="alice")
for memory_item in shared[:5]:  # Show top 5 results
    print(f"- {memory_item['memory']}")

## Update Memories

Let's add more information and see how the graph evolves:

In [ ]:
# Add more information about relationships
update_messages = [
    {
        "role": "user",
        "content": "Bob and I actually play tennis doubles together every Tuesday evening at the local club. Charlie joins us sometimes for hiking on Sundays."
    }
]

memory.add(update_messages, user_id="alice")

print("Updated memories added!")

# Search again to see enhanced connections
print("\nEnhanced tennis connections:")
enhanced_tennis = memory.search("tennis doubles Tuesday club", user_id="alice")
for memory_item in enhanced_tennis:
    print(f"- {memory_item['memory']}")

print("\nEnhanced hiking connections:")
enhanced_hiking = memory.search("Charlie hiking Sunday", user_id="alice")
for memory_item in enhanced_hiking:
    print(f"- {memory_item['memory']}")

## Clean Up

Optionally, you can delete all memories for specific users or reset the entire graph:

In [ ]:
# Delete memories for a specific user
# memory.delete_all(user_id="bob")
# print("Deleted all memories for Bob")

# Or reset the entire memory system (uncomment to use)
# memory.reset()
# print("Reset complete - all memories deleted")

print("Demo completed! FalkorDB integration is working correctly.")

## FalkorDB Advantages

FalkorDB offers several advantages for graph memory:

1. **High Performance**: FalkorDB is designed for speed, providing fast query execution and low latency.

2. **Redis Compatibility**: Uses familiar Redis commands and protocols, making it easy to integrate.

3. **Vector Support**: Native support for vector operations, perfect for similarity search with embeddings.

4. **Scalability**: Designed to handle large-scale graph workloads efficiently.

5. **Simple Deployment**: Easy to deploy using Docker or native installation.

6. **Real-time Analytics**: Supports real-time graph analytics and pattern matching.

This makes FalkorDB an excellent choice for memory graphs where performance and scalability are important considerations.